In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
import scipy.stats as st

In [ ]:
data = pd.read_parquet('data/transaction_fraud_data.parquet')
data_currency = pd.read_parquet('data/historical_currency_exchange.parquet')

Перейдём к долларам

In [ ]:
data['date'] = data['timestamp'].dt.date

In [ ]:
temp_df = data.merge(data_currency, on='date', how='left')
temp_df['amount_usd'] = temp_df.apply(
    lambda row: row['amount'] / row[row['currency']] if row['currency'] != 'USD' else row['amount'],
    axis=1
)
df = data.copy()
df['amount_usd'] = temp_df['amount_usd']

Проверка на пустые значения

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.info()

# EDA

In [ ]:
df.head(2)

# Посмотрим на доли классов

In [ ]:
show_features = [
    "country",
    "device",
    "is_outside_home_country",
    "city_size",
    "vendor_category",
    "vendor_type",
    "city",
    "channel",
    "card_type",
    "is_high_risk_vendor"
]

In [ ]:
# def show_pie(feature):
#     fig=px.pie(data_frame=df, names=feature, title=f'{feature} class ratio', color_discrete_sequence=px.colors.sequential.Viridis)
#     fig.update_traces(textposition='inside', textinfo='value+percent+label')
#     fig.show()

In [ ]:
for feature in show_features:
    show_pie(feature)

# Выводы #
В некоторых случаях присутствует явный дисбаланс классов, что 
Что очень важно: целевой класс так же в дисбалансе - мошеннических транзакций намного меньше, чем обычных

# Посмотрим на поведение целевого класса в разрезе других признаков

In [ ]:
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
show_features.append('hour')
show_features.append('day_of_week')

In [ ]:
def show_barplot(feature):
    bar_df = df.groupby(feature)['is_fraud'].mean().round(2)

    fig = px.bar(data_frame=bar_df, y=bar_df.values, x=bar_df.index, color=bar_df.index,
                 color_discrete_sequence=px.colors.sequential.Viridis,
                 title=f'Зависимость доли мошеннических операций от признака {feature}', text_auto=True)
    fig.show()


In [ ]:
df['vendor'].nunique()

In [ ]:
show_features = [
    "hour",
    "day_of_week",
    "country",
    "device",
    "is_outside_home_country",
    "city_size",
    "vendor_category",
    "vendor_type",
    "city",
    "channel",
    "card_type",
    "is_high_risk_vendor"
]

In [ ]:
for feature in show_features:
    show_barplot(feature)
    

# Выводы #
1) Мошеннические операции намного чаще производят поздней ночью
2) Активность мошенничества не зависит от дня недели, является ли категория вендора рискованной, типа карты, города (если он известен), вендора
3) Видим, что Мексика, Россия, Бразилия и Нигерия сильно выделяются на фоне остальных стран по доли мошеннических операций
4) С браузеров и телефонов намного меньше доли мошеннических операций
5) Если операция производится вне страны клиента, то высок шанс, что это мошенничество
6) В городах поменьше больше шанс, что операция мошенническая

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df, x='amount_usd', hue='is_fraud', common_norm=False, log_scale=True)
plt.title('Распределение сумм транзакций')
plt.show()


Логарифм суммы транзакции при мошеннических операциях имеет бимодальное распределение: пики на малых и крупных суммах

In [ ]:
def show_barplot_amount(feature):
    bar_df = df.groupby([feature ,'is_fraud'], as_index=False)['amount_usd'].median().round(0)

    fig = px.bar(data_frame=bar_df, y='amount_usd', x=feature, color='is_fraud',
                     color_discrete_sequence=px.colors.sequential.Viridis,
                     title=f'Зависимость размера оплаты от признаков is_fraud и {feature}', text_auto=True)
    fig.update_layout(barmode='group')
    fig.show()


In [ ]:
for feature in show_features:
    show_barplot_amount(feature)
    

Видим почти везде, что медиана суммы мошеннической транзакции больше медианы обычной 

In [ ]:
last_hour_features = ['num_transactions', 'total_amount', 'unique_merchants', 'unique_countries', 'max_single_amount']
for field in last_hour_features:
    df[f'last_hour_{field}'] = df['last_hour_activity'].apply(lambda x: x[field])

In [ ]:
for feature in last_hour_features:
    bar_df = df.groupby('is_fraud')[f'last_hour_{feature}'].mean().round(0)

    fig = px.bar(data_frame=bar_df, y=f'last_hour_{feature}', color=bar_df.index,
                         color_discrete_sequence=px.colors.sequential.Viridis,
                         title=f'Зависимость {feature} за последний час от is_fraud', text_auto=True)

    fig.show()

Мошенническая транзакция никак не влияет на активность за последний час 

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(df.corr(numeric_only=True), annot=True)

Как видим из корреляционного анализа, на признак `is_fraud` больше всего влияют `is_card_present` и `is_outside_home_country`

# Посмотрим на траты людей в разрезе других признаков

In [ ]:
def show_barplot_amount(feature):
    bar_df = df[df['is_fraud'] == False].groupby(feature)['amount_usd'].median().sort_values().round()

    fig = px.bar(data_frame=bar_df, y=bar_df.values, x=bar_df.index, color=bar_df.index,
                 color_discrete_sequence=px.colors.sequential.Viridis,
                 title=f'Зависимость медианной суммы транзакции от  признака {feature}', text_auto=True)
    fig.show()


In [ ]:
for feature in show_features:
    show_barplot_amount(feature)
    

Выше можно наблюдать, как люди тратят в разрезе различных признаков. Причем всё показано без влияния мошеннических транзакций.
# Выводы #
1) Сумма транзакции не зависит от времени, является ли категория вендора рискованной, канала проведения транзакции, города, девайса
2) Люди с типом карты "Platinum credit" намного больше тратят денег, чем люди с другими типами карты
3) Больше всего денег тратится на ритейл и путешествия
4) В большом городе люди тратятся больше
5) Нигерия сильно отстаёт от других стран по меркам трат людей

# Статистические тесты

#### Тест Уэлча

Первая гипотеза состоит в том, что суммы транзакции в мошеннических операциях в среднем больше сумм транзакции в легитимных.



Для проверки этой гипотезы будем использовать односторонний тест Уэлча с $\alpha = 0.05$

In [ ]:
first_group = df[df['is_fraud'] == True]['amount_usd']
second_group = df[df['is_fraud'] == False]['amount_usd']
st.ttest_ind(first_group, second_group, equal_var=False, alternative='greater')

p-value < $\alpha$ => отклоняем нулевую гипотезу, что значит подтверждения выдвинутой гипотезы